<a href="https://colab.research.google.com/github/jluisUdeA/Data-Validation/blob/main/BLEU_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install sacrebleu pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.8 MB/s eta 0:00:00


In [2]:
# Nombres ejemplo: ajusta a tus archivos
ref_files = ["RefES.txt"]
sys_files = ["DeepL.txt","Google.txt","Microsoft.txt"]


In [3]:
import io, os

def read_lines(path):
    with io.open(path, 'r', encoding='utf-8') as f:
        # strip() quita el último salto si lo hay; splitlines preserva alineación
        return [line.rstrip('\n\r') for line in f.read().splitlines()]

# Cargar referencias (lista de listas: [ref1_lines, ref2_lines, ...])
refs = [read_lines(p) for p in ref_files]
# Cargar sistemas
systems = {os.path.basename(p): read_lines(p) for p in sys_files}

# Verificación básica: mismos largos
n = len(refs[0])
assert all(len(r)==n for r in refs), "Todas las referencias deben tener la misma cantidad de líneas."
assert all(len(s)==n for s in systems.values()), "Todos los sistemas deben tener la misma cantidad de líneas que las referencias."
print(f"OK: {n} segmentos alineados.")

# Chequeo de líneas vacías accidentales
empty_report = {name: sum(1 for x in lines if x.strip()=="" ) for name, lines in systems.items()}
print("Líneas vacías por sistema:", empty_report)


OK: 46 segmentos alineados.
Líneas vacías por sistema: {'DeepL.txt': 0, 'Google.txt': 0, 'Microsoft.txt': 0}


In [4]:
import sacrebleu
import pandas as pd

# SacreBLEU espera refs como lista de listas (cada lista = una referencia)
# y sys como lista de strings (traducciones del sistema)
results = []

# Case-sensitive (por defecto)
for name, sys_lines in systems.items():
    bleu_cs = sacrebleu.corpus_bleu(sys_lines, refs)
    results.append({"Sistema": name, "BLEU (CS)": round(bleu_cs.score, 2)})

# Case-insensitive: lower() en todo
refs_lower = [[s.lower() for s in r] for r in refs]
for i, (name, sys_lines) in enumerate(systems.items()):
    bleu_ci = sacrebleu.corpus_bleu([s.lower() for s in sys_lines], refs_lower)
    results[i]["BLEU (CI)"] = round(bleu_ci.score, 2)

df = pd.DataFrame(results).sort_values(by="BLEU (CS)", ascending=False).reset_index(drop=True)
df


,Sistema,BLEU (CS),BLEU (CI)
0,DeepL.txt,27.04,27.35
1,Microsoft.txt,25.30,27.25
2,Google.txt,23.27,25.15


from matplotlib import pyplot as plt
df['BLEU (CS)'].plot(kind='hist', bins=20, title='BLEU (CS)')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['BLEU (CI)'].plot(kind='hist', bins=20, title='BLEU (CI)')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
df.groupby('Sistema').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df.plot(kind='scatter', x='BLEU (CS)', y='BLEU (CI)', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
df['BLEU (CI)'].plot(kind='line', figsize=(8, 4), title='BLEU (CI)')
plt.gca().spines[['top', 'right']].set_visible(False)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(df['Sistema'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(df, x='BLEU (CS)', y='Sistema', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(df['Sistema'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(df, x='BLEU (CI)', y='Sistema', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)